In [8]:
import timm
import torch
import os
import tensorflow as tf
from datasets import load_dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from datasets import load_dataset
from torch.utils.data import DataLoader
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

In [15]:
dataset = load_dataset("zh-plus/tiny-imagenet")
def preprocess_images(image, label):
    image = tf.image.resize(image, (224, 224)) # Resize images to 224x224 to match model input
    image = tf.cast(image, tf.float32) / 255.0 # Normalize pixel values
    return image, label


# Apply preprocessing
validation_dataset = dataset['valid'].map(preprocess_images).batch(32)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

TypeError: preprocess_images() missing 1 required positional argument: 'label'

In [10]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

def create_vgg19_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

vgg19_model = create_vgg19_model()
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D

def create_resnet50v2_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

resnet50v2_model = create_resnet50v2_model()
resnet50v2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
from tensorflow.keras.applications import InceptionV3

def create_inceptionv3_model(input_shape=(299, 299, 3), num_classes=200):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

inceptionv3_model = create_inceptionv3_model()
inceptionv3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator,
              epochs=epochs,
              validation_data=validation_generator)
    return model


In [14]:
# inception net requires a larger input size
_, validation_inception = preprocess_data(dataset_dir, target_size=(299, 299))
_, validation_else = preprocess_data(dataset_dir)

NameError: name 'preprocess_data' is not defined

In [38]:
vgg19_accuracy = vgg19_model.evaluate(validation_else)[1]  # Get the accuracy score
resnet50v2_accuracy = resnet50v2_model.evaluate(validation_else)[1]
inceptionv3_accuracy = inceptionv3_model.evaluate(validation_inception)[1]

print(f"VGG-19 Accuracy: {vgg19_accuracy}")
print(f"ResNet50V2 Accuracy: {resnet50v2_accuracy}")
print(f"InceptionV3 Accuracy: {inceptionv3_accuracy}")

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 200)

In [34]:
ImageDataGenerator(rescale=1./255).flow_from_directory(
            os.path.join(dataset_dir, 'val\images'),
            target_size=(224,224),
            batch_size=32,
            class_mode='categorical')

Found 0 images belonging to 0 classes.


In [16]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


len(tf.config.list_physical_devices('GPU'))